In [86]:

import spacy
import geopandas as geopd
import pandas as pd
import os
import fiona 
import shapely
from shapely.geometry import Point
from shapely.wkt import loads
import matplotlib.pyplot as plot

In [87]:
pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400

In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [115]:
from collections import Counter

In [88]:
%pip install requests
import requests

Note: you may need to restart the kernel to use updated packages.


In [91]:
#%pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sa = SentimentIntensityAnalyzer()

In [95]:
#%pip install geograpy3
import geograpy
from geograpy import extraction
from geograpy import places

c:\Users\azg526\Anaconda3\envs\GregCortez\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [96]:
%pip install geopy
from geopy.geocoders import Nominatim
import time


Note: you may need to restart the kernel to use updated packages.


El Corrido de Gregorio Cortez 
by Diane Lopez
Septemeber 8 2022
text analysis sentiment and gis project

In [97]:
#reading text file and creating a doc object by processing a string of text with the nlp object 
# Replace line breaks with spaces

#reading text file
text = open("corrido corpus\gregoriocortez_es_corrido.txt", encoding="utf-8").read()
# Replace line breaks with spaces
text_linebreaks = text.replace('\n', ' ')


In [98]:
nlp = spacy.blank("es")

In [119]:
# Created by processing a string of text with the nlp object
doc = nlp(text_linebreaks)

# Iterate over tokens in a Doc
for token in doc:
   print(token.text, token.lemma_)

En en
el el
condado condado
El el
Carmen Carmen
miren mirar
lo él
que que
ha haber
sucedido suceder
, ,
murió morir
el el
Cherife Cherife
Mayor Mayor
, ,
quedando quedar
Román Román
herido herido
. .
Se él
anduvieron andar
informando informar
como como
media medio
hora hora
después después
supieropn supieropn
que que
el el
malhechor malhechor
era ser
Gregorio Gregorio
Cortez Cortez
. .
Ya ya
insortaron insortar
a a
Cortez Cortez
por por
toditito toditito
el el
estado estado
, ,
que que
vivo vivir
o o
muerto muerto
se él
aprehenda aprehendir
proque proque
a a
various various
ha haber
matado matar
. .
Decía decir
Gregorio Gregorio
Cortez Cortez
con con
su su
pistola pistola
en en
la el
mano mano
: :
No no
siento sentir
haberlo haber él
matado matar
, ,
lo él
que que
siento sentir
es ser
a a
mi mi
hermano hermano
. .
Decía decir
Gregorio Gregorio
Cortez Cortez
con con
su su
almu almu
my my
encendid encendid
: :
No no
siento sentir
haberlo haber él
matado matar
, ,
la el
defensa defensa
es

NER Named Entity Recognition

In [137]:
#NER with Long Texts or Many Texts
import math
number_of_chunks = 80

chunk_size = math.ceil(len(text) / number_of_chunks)

text_chunks = []

for number in range(0, len(text), chunk_size):
    text_chunk = text[number:number+chunk_size]
    text_chunks.append(text_chunk)

chunked_documents = list(nlp.pipe(text_chunks))
chunked_documents

[En el condado El Carmen miren lo ,
 que ha sucedido, murió el Cherife,
  Mayor, quedando Román herido.
 Se,
  anduvieron informando como media,
  hora después supieropn que el ma,
 lhechor era Gregorio Cortez.
 Ya i,
 nsortaron a Cortez por toditito e,
 l estado, que vivo o muerto se ap,
 rehenda proque a various ha matad,
 o.
 Decía Gregorio Cortez con su p,
 istola en la mano: No siento habe,
 rlo matado, lo que siento es a mi,
  hermano.
 Decía Gregorio Cortez c,
 on su almu my encendid: No siento,
  haberlo matado, la defensa es pe,
 rmitida.
 Venían los american mas ,
 blancos que una amapola, de miedo,
  que le tenían a Cortez con su pi,
 stola.
 Decían los americanos, dec,
 ían con timidez: Vamos a seguir l,
 a huella que el malhechor es Cort,
 ez.
 Soltaron los perros juanes pa,
 'que siguieran la huella, pero al,
 canzar a Cortez.
 Tiró con rumbo a,
  Gonzales sin ninguna timidez: Sí,
 ganme, rinches cobardes, yo soy G,
 regorio Cortez.
 Se fue de Belmont,
  al rancho, lo alc

In [138]:
places = []
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "LOC":
            places.append(named_entity.text)

places_tally = Counter(places)

df = pd.DataFrame(places_tally.most_common(), columns=['place', 'count'])
df

,place,count
0,condado El Carmen,1
1,Cherife,1
2,l,1
3,ez,1
4,Belmont,1
5,Laredo,1
6,Allá,1
7,El Encinal,1
8,aquaceros,1
9,contimás de nubl,1


In [135]:
#Get NER in Context
from IPython.display import Markdown, display
import re

def get_ner_in_context(keyword, document, desired_ner_labels= False):
    
    if desired_ner_labels != False:
        desired_ner_labels = desired_ner_labels
    else:
        desired_ner_labels = ['PER', 'ORG', 'LOC']  
        
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        #process each sentence
        sentence_doc = nlp(sentence.text)
        for named_entity in sentence_doc.ents:
            #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
            if keyword.lower() in named_entity.text.lower()  and named_entity.label_ in desired_ner_labels:
                #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
                #sentence_text = sentence.text
            
                sentence_text = re.sub('\n', ' ', sentence.text)
                sentence_text = re.sub(f"{named_entity.text}", f"**{named_entity.text}**", sentence_text, flags=re.IGNORECASE)

                display(Markdown('---'))
                display(Markdown(f"**{named_entity.label_}**"))
                display(Markdown(sentence_text))

In [136]:
for document in chunked_documents:
    get_ner_in_context('Belmont', document)

---

**LOC**

 Se fue de **Belmont**

Named Entity Recongnition

In [139]:
#People
people = []

for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "PER":
            people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
df

,character,count
0,Cortez,6
1,Decía Gregorio Cortez,4
2,Gregorio Cortez,3
3,Cort,2
4,Román,1
5,rehenda proque,1
6,Decía Gregorio Cortez c,1
7,ían,1
8,Tiró,1
9,Gonzales,1


Part of Speech

In [141]:
for token in doc:
    print(token.lemma_, token.pos_, token.dep_)

en ADP case
el DET det
condado NOUN obl
el DET det
Carmen PROPN nsubj
mirar VERB ROOT
él PRON det
que PRON nsubj
haber AUX aux
suceder VERB ccomp
, PUNCT punct
morir VERB advcl
el DET det
Cherife PROPN nsubj
Mayor PROPN flat
, PUNCT punct
quedar VERB advcl
Román PROPN nsubj
herido ADJ amod
. PUNCT punct
él PRON obj
andar VERB ROOT
informar VERB xcomp
como SCONJ mark
medio NUM nummod
hora NOUN obl
después ADV advmod
supieropn PROPN obj
que SCONJ mark
el DET det
malhechor NOUN nsubj
ser AUX cop
Gregorio PROPN ccomp
Cortez PROPN flat
. PUNCT punct
ya ADV advmod
insortar VERB ROOT
a ADP case
Cortez PROPN obj
por ADP advmod
toditito PROPN fixed
el DET det
estado NOUN nsubj
, PUNCT punct
que PRON nsubj
vivir VERB acl
o CCONJ cc
muerto ADJ conj
él PRON obj
aprehendir VERB ccomp
proque ADJ obj
a ADP case
various NOUN obj
haber AUX aux
matar VERB advcl
. PUNCT punct
decir VERB ROOT
Gregorio PROPN nsubj
Cortez PROPN flat
con ADP case
su DET det
pistola NOUN obl
en ADP case
el DET det
mano NOUN o

Keyword Extration

In [142]:
import re
from IPython.display import Markdown, display

In [147]:
def find_sentences_with_keyword(keyword, doc):
    
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in doc.sents:
        sentence = sentence.text
        
        #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
        if keyword.lower() in sentence.lower():
            
            #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            sentence = re.sub('\n', ' ', sentence)
            sentence = re.sub(f"{keyword}", f"**{keyword}**", sentence, flags=re.IGNORECASE)

            display(Markdown(sentence))
            

In [149]:
find_sentences_with_keyword(keyword="Laredo", doc=doc)


Salió Gregorio Cortez, salió con rumbo a **Laredo**, no lo quisieron seguir porque le tuvieron miedo.

In [154]:
#Make a list of tokens and POS labels from document if the token is a word 
tokens_and_labels = [(token.text, token.pos_) for token in doc if token.is_alpha]

In [155]:
#Make a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    #Loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):
        
        #Index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        #Append this word combo to the master list "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [156]:
bigrams = get_bigrams(tokens_and_labels)

In [157]:
bigrams[5:20]

[[('miren', 'VERB'), ('lo', 'PRON')],
 [('lo', 'PRON'), ('que', 'PRON')],
 [('que', 'PRON'), ('ha', 'AUX')],
 [('ha', 'AUX'), ('sucedido', 'VERB')],
 [('sucedido', 'VERB'), ('murió', 'VERB')],
 [('murió', 'VERB'), ('el', 'DET')],
 [('el', 'DET'), ('Cherife', 'PROPN')],
 [('Cherife', 'PROPN'), ('Mayor', 'PROPN')],
 [('Mayor', 'PROPN'), ('quedando', 'VERB')],
 [('quedando', 'VERB'), ('Román', 'PROPN')],
 [('Román', 'PROPN'), ('herido', 'ADJ')],
 [('herido', 'ADJ'), ('Se', 'PRON')],
 [('Se', 'PRON'), ('anduvieron', 'VERB')],
 [('anduvieron', 'VERB'), ('informando', 'VERB')],
 [('informando', 'VERB'), ('como', 'SCONJ')]]

In [158]:
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]        
        
        #Check to see if keyword is in the bigram
        if keyword in words:
            
            for word, label in bigram:
                
                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        neighbor_words.append(word.lower())
    
    return Counter(neighbor_words).most_common()

In [160]:
get_neighbor_words("Cortez", bigrams)


[('gregorio', 12),
 ('a', 4),
 ('con', 4),
 ('ya', 2),
 ('se', 2),
 ('por', 1),
 ('es', 1),
 ('soltaron', 1),
 ('tiró', 1),
 ('salió', 1),
 ('pa', 1),
 ('echando', 1),
 ('dicen', 1),
 ('convieniente', 1),
 ('le', 1),
 ('case', 1),
 ('llorar', 1),
 ('entrega', 1),
 ('les', 1),
 ('decía', 1)]

In [302]:
greg_df = pd.read_fwf('log.csv')
greg_df


,"title,intro"
0,"En el condado de El Carmen miren lo que ha sucedido, murió el Cherife Mayor, quedando Román herido."
1,"En el condado de El Carmen tal desgracia sucedio, murió el Cherife Mayor, no saben quien lo mato."
2,Se anduvieron informando como media hora después supieropn que el malhechor era Gregorio Cortez.
3,"Ya insortaron a Cortez por toditito el estado, que vivo o muerto se aprehenda proque a various ha matado."
4,"Decía Gregorio Cortez con su pistola en la mano: No siento haberlo matado, lo que siento es a mi hermano."
5,"Decía Gregorio Cortez con su almu my encendid: No siento haberlo matado, la defensa es permitida."
6,"Venían los american mas blancos que una amapola, de miedo que le tenían a Cortez con su pistola."
7,"Decían los americanos, decían con timidez: Vamos a seguir la huella que el malhechor es Cortez."
8,"Soltaron los perros juanes pa'que siguieran la huella, pero alcanzar a Cortez."
9,"Tiró con rumbo a Gonzales sin ninguna timidez: Síganme, rinches cobardes, yo soy Gregorio Cortez."
